# Resolving the Dependencies
Installing the Amaranth HLD toolchain

In [1]:
!pip install amaranth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 KB 2.4 MB/s eta 0:00:00


# Implementing a up counter
Creating a up counter that when *en* is asserted if *ovf* is asserted the counter receive zero and if *en* is asserted and *ovf* is not asserted the counter receive `counter = counter + 1`

In [2]:
from amaranth import *

class upCounter(Elaboratable):
  def __init__ (self, limit):
    self.limit = limit

    #Ports
    self.en = Signal()
    self.ovf = Signal()

    #State
    self.count = Signal()

  def elaborate(self, plataform):
    m = Module()

    m.d.comb += self.ovf.eq(self.count == self.limit)

    with m.If(self.en):
      with m.If(self.ovf):
        m.d.sync += self.count.eq(0)
      with m.Else():
        m.d.sync += self.count.eq(self.count + 1)
      
    return m

# Testing the up counter

In [3]:
from amaranth.sim import Simulator

dut = upCounter(25)

def bench():
  yield dut.en.eq(0)
  for _ in range(30):
    yield
    assert not (yield dut.ovf)

    yield dut.en.eq(1)
    for _ in range(25):
      yield
      assert (yield dut.ovf)
    yield
    assert (yield dut.ovf)

    yield
    assert not (yield dut.ovf)

    sim = Simulator(dut)
    sim.add_clock(1e-6)
    sim.add_sync_process(bench)
    with sim.write_vcd("up_counter.vcd"):
      sim.run()